In [27]:
import numpy 
import pandas as pd

In [41]:
data = pd.read_csv('./train.csv', encoding='latin-1', delimiter=',', quotechar='"')
print(data.shape)
data.head()

(99781, 3)


,ItemID,Sentiment,SentimentText;;;;;;;;
0,1,0.0,is so sad for my APL frie...
1,2,0.0,I missed the New Moon trail...
2,3,1.0,omg its already 7:30 :O;;;;;;;;
3,4,0.0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0.0,i think mi bf is cheating on me!!! ...


In [42]:
data = data.dropna()

In [44]:
# Obtener la distribución de clases
class_distribution = data['Sentiment'].value_counts()

# Imprimir la distribución de clases
print(class_distribution)

1.0    41615
0.0    33424
Name: Sentiment, dtype: int64


In [45]:
minority_class_size = min(class_distribution)

# Undersampling de la clase mayoritaria
undersampled_df = pd.concat([
    data[data['Sentiment'] == 0],
    data[data['Sentiment'] == 1].sample(minority_class_size, replace=True)
])

new_class_distribution = undersampled_df['Sentiment'].value_counts()

In [46]:
new_class_distribution

0.0    33424
1.0    33424
Name: Sentiment, dtype: int64

In [47]:
data =  data.rename(columns={'SentimentText;;;;;;;;': 'SentimentText'})


In [48]:
data.columns

Index(['ItemID', 'Sentiment', 'SentimentText'], dtype='object')

In [49]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Dividir los datos en características (X) y etiquetas (y)
X = data['SentimentText']
y = data['Sentiment']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear un vectorizador TF-IDF
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Crear y entrenar el modelo de Bosque Aleatorio
rf_model = RandomForestClassifier()
rf_model.fit(X_train_vectorized, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = rf_model.predict(X_test_vectorized)

# Evaluar el rendimiento del modelo
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.73      0.71      0.72      6729
         1.0       0.77      0.79      0.78      8279

    accuracy                           0.75     15008
   macro avg       0.75      0.75      0.75     15008
weighted avg       0.75      0.75      0.75     15008



In [53]:
text = "I can't believe how poorly this game is designed. The controls are clunky, the graphics are outdated, and the gameplay is incredibly boring. I regret wasting my money on it."
new_text_vectorized = vectorizer.transform([text])



In [55]:
# Realizar la predicción del sentimiento del nuevo texto
predicted_sentiment = rf_model.predict(new_text_vectorized)[0]

# Imprimir el resultado de la clasificación
if predicted_sentiment == 0:
    print("El sentimiento del texto es negativo.")
else:
    print("El sentimiento del texto es positivo.")

El sentimiento del texto es positivo.
